In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pandas import DataFrame
from pandas import concat
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from scipy.stats import kde

In [ ]:
data = pd.read_csv('D:\dissertation 10th sem\HomeC.csv', low_memory=False)
print(data)

In [ ]:
data = data[0:-1]
data.tail()

In [ ]:
data['sum_Furnace'] = data[['Furnace 1 [kW]','Furnace 2 [kW]']].sum(axis=1)
data['avg_Kitchen'] = data[['Kitchen 12 [kW]','Kitchen 14 [kW]','Kitchen 38 [kW]']].mean(axis=1)

In [ ]:
data = data.drop(['Kitchen 12 [kW]','Kitchen 14 [kW]','Kitchen 38 [kW]'], axis=1)
data = data.drop(['Furnace 1 [kW]','Furnace 2 [kW]'], axis=1)
data.columns

In [ ]:
time_index = pd.date_range('2016-01-01 08:00', periods=len(data),  freq='min')  
time_index = pd.DatetimeIndex(time_index)
data = data.set_index(time_index)
data.head(3)

In [ ]:
data.info()

In [ ]:
data['cloudCover']

In [ ]:
#data[data['cloudCover']=='cloudCover'].shape

In [ ]:
data['cloudCover'].replace(['cloudCover'], method='bfill', inplace=True)
data['cloudCover']=data['cloudCover'].astype('float')
data['cloudCover']

In [ ]:
data.drop(['use [kW]','gen [kW]','summary'], axis=1, inplace=True)
data.head(3)

In [ ]:
data.info()

In [ ]:
home_dat = data.select_dtypes(exclude=['object'])
home_dat.head(3)

In [ ]:
energy_data = home_dat.filter(items=['Solar [kW]','Dishwasher [kW]','Home office [kW]','Fridge [kW]','Wine cellar [kW]',
                                     'Garage door [kW]', 'Barn [kW]','Well [kW]','Microwave [kW]','Living room [kW]',
                                     'sum_Furnace', 'avg_Kitchen'])

weather_data = home_dat.filter(items=['temperature','humidity','visibility','apparentTemperature','pressure','windSpeed',
                                  'cloudCover','windBearing','precipIntensity','dewPoint','precipProbability'])

In [ ]:
energy_data.head(3)

In [ ]:
weather_data.head(3)

In [ ]:
energy_per_day = energy_data.resample('D').sum()
energy_per_day.head(3)

In [ ]:
energy_per_month = energy_data.resample('M').sum()
energy_per_month.head(3)

In [ ]:
plt.figure(figsize=(20,10)) 
sns.lineplot(data= energy_per_month.filter(items=['Dishwasher [kW]','Home office [kW]','Fridge [kW]','Garage door [kW]',
                                                  'Wine cellar [kW]', 'Barn [kW]', 'Microwave [kW]',
                                                  'Living room [kW]']) , dashes=False  )
#Home office has highest consumption
#Microwave has lowest consumption
#Wine cellar shows a peak in september
#Microwave, Garage door & Dishwasher energy consumption is constant irrespective of weather as their usage is directly 
 #linked to human movements 

In [ ]:
weather_per_day = weather_data.resample('D').mean()  # note!! (mean) # D =>> for day sample
weather_per_day.head(3)                

In [ ]:
weather_per_month = weather_data.resample('M').mean()   # M =>> for month sample
weather_per_month.head(3)

In [ ]:
plt.figure(figsize=(20,10)) 
sns.lineplot(data= weather_per_month.filter(items=['temperature','humidity','visibility','apparentTemperature','pressure',
                                                   'windSpeed','cloudCover','windBearing','precipIntensity','dewPoint',
                                                   'precipProbability']) , dashes=False)

In [ ]:
rooms_energy = energy_per_month.filter(items=[      # remove the devices consumption 
                                     'Home office [kW]', 'Wine cellar [kW]', 'avg_Kitchen','Barn [kW]', 'Living room [kW]']) 
devices_energy = energy_per_month.filter(items=[    # remove the rooms consumption
                                     'Dishwasher [kW]', 'sum_Furnace', 'Garage door [kW]', 'Well [kW]',
                                     'Microwave [kW]'])

all_rooms_consum = rooms_energy.sum()
all_devices_consum = devices_energy.sum()
print(all_rooms_consum)
print(all_devices_consum)

In [ ]:
plot = all_rooms_consum.plot(kind="pie", autopct='%.2f', figsize=(10,10))
plot.set_title("Consumption for rooms")
plot.set_ylabel('%')

In [ ]:
plot = all_devices_consum.plot(kind="pie", autopct='%.2f', figsize=(10,10))
plot.set_title("Consumption for devices")
plot.set_ylabel('%')

In [ ]:
fig,ax = plt.subplots(figsize=(10, 8)) 
corr = data[['Dishwasher [kW]','Home office [kW]','Fridge [kW]','Wine cellar [kW]','Garage door [kW]','Barn [kW]',
             'Well [kW]','Microwave [kW]','Living room [kW]','sum_Furnace','avg_Kitchen']].corr()
sns.heatmap(corr, annot=True, vmin=-1.0, vmax=1.0, center=0)
ax.set_title('Correlation of Appliances',size=20)
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(10, 8)) 
corr = data[['temperature','apparentTemperature','humidity','visibility','pressure','windSpeed','cloudCover',
           'precipIntensity','dewPoint']].corr()
sns.heatmap(corr, annot=True, vmin=-1.0, vmax=1.0, center=0)
ax.set_title('Correlation of Weather Information',size=20)
plt.show()

# Temperature is related to apparentTemperature and dewPoint.
# Humidity is related to visibility, windSpeed, cloudCover and dewPoint.
# Visibility is related to humidity, windSpeed, cloudCover and precipIntensity.
# CloudCover is related to humidity, visibility and precipIntensity.
# PrecipIntensity is related to visibility and cloudCover.
# DewPoint is related to temperature, apparentTemperature and humidity.

In [ ]:
fig,ax = plt.subplots(figsize=(20, 12)) 
corr = data[['Dishwasher [kW]','Home office [kW]','Fridge [kW]','Wine cellar [kW]','Garage door [kW]','Barn [kW]',
             'Well [kW]','Microwave [kW]','Living room [kW]','sum_Furnace','avg_Kitchen','temperature','apparentTemperature',
             'humidity','visibility','pressure','windSpeed','cloudCover','precipIntensity','dewPoint']].corr()
sns.heatmap(corr, annot=True, vmin=-1.0, vmax=1.0, center=0)
ax.set_title('Correlation of Appliances & Weather Information',size=20)
plt.show()

# Fridge is related to temperature, apparentTemperature and dewPoint.
# Wine cellar is related to temperature, apparentTemperature and dewPoint.
# sum_Furnace is related to temperature, apparentTemperature, windSpeed and dewPoint.

In [ ]:
weather=data['icon'].unique()
weather

In [ ]:
data['icon'].map({'clear-night':0,
                  'partly-cloudy-night':1, 'clear-day':2, 'cloudy':3,
       'partly-cloudy-day':4, 'rain':5, 'snow':6, 'wind':7, 'fog':8})

In [ ]:
dummy=pd.get_dummies(data['icon'])


In [ ]:
X=data.drop(['House overall [kW]','time','icon'],axis=1)
X=X.join(dummy)
Y=data['House overall [kW]']

regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(X,Y)

In [ ]:
r=regressor.feature_importances_

In [ ]:
fi=pd.Series(r,index=X.columns)

In [ ]:
fi['icon']=fi[weather].sum()
fi=fi.drop(weather)

In [ ]:
fi.sum()
fi

In [ ]:
def series_to_supervised(data, n_in, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

data_dw=data.resample('D').mean()


In [ ]:
category=fi.index[0:len(fi)-1]

In [ ]:
spam_score=pd.DataFrame([],columns=['m1','m2','m3','m4'],index=category)

xb1=XGBRegressor(n_estimators=999,max_depth=9)
xb2=DecisionTreeRegressor(max_depth=9)
xb3=RandomForestRegressor(max_depth=9, max_features=10)
xb4=GradientBoostingRegressor(learning_rate=0.1, max_features=12)

for col in category:
    coldata=data_dw[col]
    coldata=list((coldata-coldata.mean())/coldata.std())
    dw_dataset=series_to_supervised(coldata,12,1)

    x=dw_dataset.drop('var1(t)',axis=1)
    y=dw_dataset['var1(t)']
    x_train,x_test,Y_train,Y_test=train_test_split(x, y, test_size=0.2)
    
    
    xb1.fit(x_train, Y_train,eval_set=[(x_train, Y_train), (x_test, Y_test)])
    y_pred1 = xb1.predict(x_test)
    x1= np.sqrt(metrics.mean_squared_error(Y_test, y_pred1))
    spam_score.loc[col,'m1']=x1*fi[col]
    
    xb2.fit(x_train, Y_train)
    y_pred2 = xb2.predict(x_test)
    x2= np.sqrt(mean_squared_error(Y_test, y_pred2))
    spam_score.loc[col,'m2']=x2*fi[col]
    
    xb3.fit(x_train, Y_train)
    y_pred3 = xb3.predict(x_test)
    x3= np.sqrt(mean_squared_error(Y_test, y_pred3))
    spam_score.loc[col,'m3']=x3*fi[col]
    

    xb4.fit(x_train, Y_train)
    y_pred4 = xb4.predict(x_test)
    x4= np.sqrt(mean_squared_error(Y_test, y_pred4))
    spam_score.loc[col,'m4']=x4*fi[col]
    

In [ ]:
spam_score

In [ ]:
x=spam_score['m1'].head(10)
y=spam_score['m1'].tail(13)
ax = x.plot.kde(color="red")      #House Appliances
ax = y.plot.kde(color="blue")     #Weather Conditions

In [ ]:
x=spam_score['m2'].head(10)
y=spam_score['m2'].tail(13)
ax = x.plot.kde(color="red")         #House Appliances
ax = y.plot.kde(color="blue")        #Weather Conditions

In [ ]:
x=spam_score['m3'].head(10)
y=spam_score['m3'].tail(13)
ax = x.plot.kde(color="red")          #House Appliances     
ax = y.plot.kde(color="blue")         #Weather Conditions 

In [ ]:
x=spam_score['m4'].head(10)
y=spam_score['m4'].tail(13)
ax = x.plot.kde(color="red")             #House Appliances 
ax = y.plot.kde(color="blue")            #Weather Conditions

In [ ]:
spam_score1=pd.DataFrame([],columns=['l'],index=category)

x_train2=x_train.values.reshape(x_train.shape[0],x_train.shape[1],1)
    # x_train2.shape
x_test2=x_test.values.reshape(x_test.shape[0],x_test.shape[1],1)
    # x_test2.shape

model = Sequential()
model.add(LSTM(64, input_shape=(x_train.shape[1], 1)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

for col in category:
    coldata=data_dw[col]
    coldata=list((coldata-coldata.min())/(coldata.max()-coldata.min()))
    dw_dataset=series_to_supervised(coldata,12,1)

    x=dw_dataset.drop('var1(t)',axis=1)
    y=dw_dataset['var1(t)']
    x_train,x_test,Y_train,Y_test=train_test_split(x, y, test_size=0.2)
   
    
    history = model.fit(x_train2, Y_train, epochs=70, batch_size=30, validation_data=(x_test2,Y_test), verbose=2, shuffle=False)
    y_pred5=model.predict(x_test2)
    x5= np.sqrt(metrics.mean_squared_error(Y_test, y_pred5))
    spam_score1.loc[col,'l']=x5*fi[col]
       

    

In [ ]:
spam_score1

In [ ]:
print(type(spam_score1))

In [ ]:
x=spam_score1['l'].head(10)
y=spam_score1['l'].tail(13)
ax = x.plot.kde(color="red")          #House Appliances
ax = y.plot.kde(color="blue")         #Weather Conditions